<a href="https://colab.research.google.com/github/lamalex/cs722-pointnet/blob/main/cs722-pointnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -L http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip -o ModelNet10.zip
!unzip -q ModelNet10.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  451M  100  451M    0     0  75.1M      0  0:00:06  0:00:06 --:--:-- 78.2M


In [ ]:
!python -V

Python 3.6.9


In [110]:
import random
import numpy as np
from pathlib import Path
import plotly.graph_objects as go
import plotly.express as px

**ModelNet10** contains CAD models from 10 categories. The models are described using .off files.
.off is a simple format where:
- The first line has OFF to mark the file as .off
- The 2nd line is # vertices, # faces, # edges
- List of vertices (X, Y, Z, W)
- List of faces
- List of edges

In [111]:
path = Path("ModelNet10")

In [114]:
def read_off(file):
    if 'OFF' != file.readline().strip():
        raise('Not a valid OFF header')
    n_verts, n_faces, _ = tuple([int(s) for s in file.readline().strip().split(' ')])
    verts = [[float(s) for s in file.readline().strip().split(' ')] for _ in range(n_verts)]
    faces = [[int(s) for s in file.readline().strip().split(' ')][1:] for _ in range(n_faces)]
    return verts, faces

## Polygon Mesh representation of Chair

In [115]:
with open(f'{path}/chair/train/chair_0001.off', 'r') as night_stand_file:
    mesh = read_off(night_stand_file)

fig = go.Figure(data=
    go.Mesh3d(
        x=[x[0] for x in mesh[0]],
        y=[y[1] for y in mesh[0]],
        z=[z[2] for z in mesh[0]],
        i=[i[0] for i in mesh[1]],
        j=[j[1] for j in mesh[1]],
        k=[k[2] for k in mesh[1]],
    ))
fig.show()


# The same chair with only vertices



In [116]:
fig = px.scatter_3d(
        x=[x[0] for x in mesh[0]],
        y=[y[1] for y in mesh[0]],
        z=[z[2] for z in mesh[0]],
    )
fig.show()

# The same chair as a PointCloud
The CAD representation is face and vertices, but we can sample
faces to generate a point cloud.

In [117]:
verts, faces = mesh
areas = np.zeros((len(faces)))
verts = np.array(verts)

# function to calculate triangle area by its vertices
# https://en.wikipedia.org/wiki/Heron%27s_formula
def triangle_area(pt1, pt2, pt3):
    side_a = np.linalg.norm(pt1 - pt2)
    side_b = np.linalg.norm(pt2 - pt3)
    side_c = np.linalg.norm(pt3 - pt1)
    s = 0.5 * ( side_a + side_b + side_c)
    return max(s * (s - side_a) * (s - side_b) * (s - side_c), 0)**0.5

# we calculate areas of all faces in our mesh
for i in range(len(areas)):
    areas[i] = (triangle_area(verts[faces[i][0]],
                              verts[faces[i][1]],
                              verts[faces[i][2]]))

k = 3000
# we sample 'k' faces with probabilities proportional to their areas
# weights are used to create a distribution.
# they don't have to sum up to one.
sampled_faces = (random.choices(faces, 
                                weights=areas,
                                k=k))

# function to sample points on a triangle surface
def sample_point(pt1, pt2, pt3):
    # barycentric coordinates on a triangle
    # https://mathworld.wolfram.com/BarycentricCoordinates.html
    s, t = sorted([random.random(), random.random()])
    f = lambda i: s * pt1[i] + (t-s) * pt2[i] + (1-t) * pt3[i]
    return (f(0), f(1), f(2))
 
pointcloud = np.zeros((k, 3))

# sample points on chosen faces for the point cloud of size 'k'
for i in range(len(sampled_faces)):
    pointcloud[i] = (sample_point(verts[sampled_faces[i][0]],
                                  verts[sampled_faces[i][1]],
                                  verts[sampled_faces[i][2]]))

In [118]:
fig = px.scatter_3d(
    x=[x[0] for x in pointcloud],
    y=[y[1] for y in pointcloud],
    z=[z[2] for z in pointcloud],
)
fig.show()

# T-Net

In [2]:
from itertools import islice
# from functools import reduce

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

def window(seq, n=2):
    '''
    Returns a sliding window (of width n) over data from the iterable
    s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   
    '''
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result


class TNet(nn.Module):
    '''
    TNet is a regression network for predicting a k x k transformation matrix
    '''
    def __init__(self, k: int):
        super(TNet, self).__init__()
        dims = [k, 64, 128, 1024, 512, 256, k**2]
        cnn_dims = window(dims[:4])
        fc_dims = window(dims[3:])
        
        self.k = k
        self.cnn = [nn.Conv1d(i, o, 1) for (i, o) in cnn_dims]
        self.fc = [nn.Linear(i, o) for (i, o) in fc_dims]
        self.bn = [nn.BatchNorm1d(d) for d in dims[1:-1]]

    def forward(self, x):
        batchsize = x.size()[0]
        bn_iter = iter(self.bn)

        # Extremely obtuse one-liner for sequential application of ReLU(BatchNorm(CNN)) for each CNN layer
        # But I finally found a use for the walrus operator so I'm keeping it.
        # EXCEPT OMFG GOOGLE COLAB IS PYTHON 3.6 SO I CAN'T USE WALRUS OPERATOR := 🤬
        # x = reduce(lambda x, f: (cnn := f[0], bn := f[1], F.relu(bn(cnn(x))))[-1], zip(self.cnn, bn_iter), x)
        for i in range(self.cnn):
            bn = bn_iter.__next__()
            x = F.relu(bn(self.cnn[i](x)))
        x = torch.max(x, 2, keepdim = True)[0]
        x = x.view(-1, 1024)

        # IT STINGS EVERY TIME
        # x = reduce(lambda x, f: (fc := f[0], bn := f[1], F.relu(bn(fc(x))))[-1], zip(self.fc, bn_iter), x)
        for i in range(self.fc[:-1]):
            bn = bn_iter.__next__()
            x = F.relu(bn(self.fc[i](x)))
        
        x = self.fc[-1](x)
        ident = Variable(torch.from_numpy(np.identity(self.k))).view(1, self.k**2).repeat(batchsize, 1)
        ident = ident.cuda()

        x.view(-1, self.k, self.k)
        x += ident

        return x